In [64]:
import csv
import collections
import numpy as np

In [86]:
data_dict_extra_years = []
with open('plays_sorted.csv', 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    for line in enumerate(reader):
        data_dict_extra_years.append(line[1])

In [66]:
mapper = {}
mapper['ARI'] = 'Cardinals'
mapper['ATL'] = 'Falcons'
mapper['BAL'] = 'Ravens'
mapper['BUF'] = 'Bills'
mapper['CAR'] = 'Panthers'
mapper['CHI'] = 'Bears'
mapper['CIN'] = 'Bengals'
mapper['CLE'] = 'Browns'
mapper['DAL'] = 'Cowboys'
mapper['DEN'] = 'Broncos'
mapper['DET'] = 'Lions'
mapper['GB'] = 'Packers'
mapper['HOU'] = 'Texans'
mapper['IND'] = 'Colts'
mapper['JAC'] = 'Jaguars'
mapper['KC'] = 'Chiefs'
mapper['MIA'] = 'Dolphins'
mapper['MIN'] = 'Vikings'
mapper['NE'] = 'Patriots'
mapper['NO'] = 'Saints'
mapper['NYG'] = 'Giants'
mapper['NYJ'] = 'Jets'
mapper['OAK'] = 'Raiders'
mapper['PHI'] = 'Eagles'
mapper['PIT'] = 'Steelers'
mapper['SD'] = 'Chargers'
mapper['SEA'] = 'Seahawks'
mapper['SF'] = '49ers'
mapper['STL'] = 'Rams'
mapper['TB'] = 'Buccaneers'
mapper['TEN'] = 'Titans'
mapper['WAS'] = 'Redskins'

In [87]:
data_dict = []
for datum in data_dict_extra_years:
    game_id = datum['gameid']
    year_loc = game_id.rfind('/')+1
    year = game_id[year_loc:year_loc+2]
    
    if year_loc==0:
        date = (('%s/%s') %(int(game_id[4:6]), int(game_id[6:8]))) + '/' +game_id[2:4]
        teamname_loc = game_id.rfind('_')+1
        away_team = mapper[game_id[teamname_loc:game_id.rfind('@')]]
        home_team = mapper[game_id[game_id.rfind('@')+1:]]
        datum['gameid'] = date+'_'+away_team+home_team
        data_dict.append(datum)
    elif (year != '15' and year !='16'):
        datum['gameid'] = game_id[:year_loc+2]+'_'+game_id[year_loc+2:]
        data_dict.append(datum)
        
#removed 2015/2016 years
#added underscore between date and teams

In [88]:
data_dict[54313]

{'def': 'Seahawks',
 'defscore': '17',
 'down': '2',
 'gameid': '9/29/02_VikingsSeahawks',
 'off': 'Vikings',
 'offscore': '7',
 'time': '2318',
 'togo': '5',
 'ydline': '52'}

In [90]:
winners_dict = []
for year in range(2002, 2015):
    with open('nfl_results-master/nfl %s.csv' % year, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        for line in enumerate(reader):
            winners_dict.append(line[1])

In [100]:
winners_dict[0]

{'home_score': '13',
 'home_team': 'Giants',
 'kickoff': '2002-09-05T00:00:00+00:00',
 'season': '2002',
 'visiting_team': '49ers',
 'visitors_score': '16',
 'week': '1'}

In [113]:
winners = {}
for game in winners_dict:
    kickoff = game['kickoff']
    date = ('%s/%s/' % (int(kickoff[5:7]), int(kickoff[8:10])))+kickoff[2:4]
    gameid1 = date + '_'+ game['visiting_team'] + game['home_team']
    winners[gameid1] = game['visiting_team'] if game['visitors_score'] > game['home_score'] else game['home_team']
    gameid2 = date + '_' + game['home_team'] +game['visiting_team']
    winners[gameid2] = game['visiting_team'] if game['visitors_score'] > game['home_score'] else game['home_team']

In [124]:
new_data_dict = []
for datum in data_dict:
    if datum['gameid'] in winners:
        if (winners[datum['gameid']]==datum['off']):
            datum['won_game'] = '1'
        else:
            datum['won_game'] = '0'
        new_data_dict.append(datum)

In [125]:
headers = ['gameid', 'off', 'def', 'offscore', 'defscore', 'down', 'togo', 'ydline', 'time', 'won_game']

In [126]:
data = []
for play in new_data_dict:
    row = []
    for feature in headers:
        if play[feature]:
            row.append(play[feature])
    data.append(row)

In [127]:
with open('playsandwins.csv','w') as out:
    writer = csv.writer(out)
    writer.writerow(headers)
    for line in data:
        writer.writerow(line)